//+------------------------------------------------------------------+
//|                                         H1_12PM_RSI_EA.mq5       |
//+------------------------------------------------------------------+
#property strict

#include <Trade/Trade.mqh>
CTrade trade;

//--- inputs
input double RiskPercent = 1.0;    // Risk per trade in %
input int    RSIPeriod  = 14;      // RSI period
input double Overbought = 70.0;    // RSI overbought
input double Oversold   = 30.0;    // RSI oversold

//--- global
bool tradesPlaced = false;
int rsiHandle = -1;

//+------------------------------------------------------------------+
//| Expert initialization                                            |
//+------------------------------------------------------------------+
int OnInit()
{
   tradesPlaced = false;

   // Create RSI handle
   rsiHandle = iRSI(_Symbol, PERIOD_H1, RSIPeriod, PRICE_CLOSE);
   if(rsiHandle < 0)
   {
      Print("Failed to create RSI handle: ", GetLastError());
      return(INIT_FAILED);
   }

   return(INIT_SUCCEEDED);
}

//+------------------------------------------------------------------+
//| Expert tick function                                             |
//+------------------------------------------------------------------+
void OnTick()
{
   // Only run on H1 chart
   if(Period() != PERIOD_H1)
      return;

   int last = iBars(_Symbol, PERIOD_H1) - 2; // previous closed candle
   if(last < 0) return;

   datetime t_candle = iTime(_Symbol, PERIOD_H1, last);
   MqlDateTime t;
   TimeToStruct(t_candle, t);

   // Trigger only once per day for 12 PM candle
   if(t.hour == 12 && t.min == 0 && !tradesPlaced)
   {
      double high  = iHigh(_Symbol, PERIOD_H1, last);
      double low   = iLow(_Symbol, PERIOD_H1, last);
      double mid   = (high + low) / 2.0;
      double range = high - low;

      //--- calculate lot size for 1% risk
      double tickValue = SymbolInfoDouble(_Symbol, SYMBOL_TRADE_TICK_VALUE);
      double tickSize  = SymbolInfoDouble(_Symbol, SYMBOL_TRADE_TICK_SIZE);
      if(tickValue <= 0 || tickSize <= 0) return;
      double slPoints  = (high - mid) / tickSize;
      double riskMoney = AccountInfoDouble(ACCOUNT_BALANCE) * RiskPercent / 100.0;
      double lotSize   = riskMoney / (slPoints * tickValue);
      lotSize = MathMax(lotSize, SymbolInfoDouble(_Symbol, SYMBOL_VOLUME_MIN));

      //--- place BuyStop
      MqlTradeRequest buyRequest;
      MqlTradeResult  buyResult;
      ZeroMemory(buyRequest);
      ZeroMemory(buyResult);

      buyRequest.action       = TRADE_ACTION_PENDING;
      buyRequest.symbol       = _Symbol;
      buyRequest.type         = ORDER_TYPE_BUY_STOP;
      buyRequest.price        = high;
      buyRequest.sl           = mid;
      buyRequest.tp           = 0; // TP managed by RSI
      buyRequest.volume       = lotSize;
      buyRequest.deviation    = 5;
      buyRequest.type_filling = ORDER_FILLING_FOK;

      if(!OrderSend(buyRequest, buyResult))
         Print("BuyStop failed: ", GetLastError());

      //--- place SellStop
      MqlTradeRequest sellRequest;
      MqlTradeResult  sellResult;
      ZeroMemory(sellRequest);
      ZeroMemory(sellResult);

      sellRequest.action       = TRADE_ACTION_PENDING;
      sellRequest.symbol       = _Symbol;
      sellRequest.type         = ORDER_TYPE_SELL_STOP;
      sellRequest.price        = low;
      sellRequest.sl           = mid;
      sellRequest.tp           = 0; // TP managed by RSI
      sellRequest.volume       = lotSize;
      sellRequest.deviation    = 5;
      sellRequest.type_filling = ORDER_FILLING_FOK;

      if(!OrderSend(sellRequest, sellResult))
         Print("SellStop failed: ", GetLastError());

      tradesPlaced = true;

      //--- optional: highlight the 12 PM candle
      string objName = "H1_12PM_" + IntegerToString(t_candle);
      if(!ObjectCreate(0, objName, OBJ_RECTANGLE, 0, t_candle, low, t_candle + PeriodSeconds(), high))
         Print("Failed to draw candle highlight: ", GetLastError());
      else
      {
         ObjectSetInteger(0, objName, OBJPROP_COLOR, clrYellow);
         ObjectSetInteger(0, objName, OBJPROP_STYLE, STYLE_SOLID);
         ObjectSetInteger(0, objName, OBJPROP_WIDTH, 2);
      }
   }

   //--- manage open positions with RSI-based TP
   if(PositionsTotal() == 0) return;

   double rsiBuffer[];
   if(CopyBuffer(rsiHandle, 0, 0, 1, rsiBuffer) <= 0) return;
   double rsiValue = rsiBuffer[0];

   for(int i = PositionsTotal()-1; i >= 0; i--)
   {
      ulong ticket = PositionGetTicket(i);
      if(PositionSelectByTicket(ticket))
      {
         string sym  = PositionGetString(POSITION_SYMBOL);
         ENUM_POSITION_TYPE type = (ENUM_POSITION_TYPE)PositionGetInteger(POSITION_TYPE);

         if(type == POSITION_TYPE_BUY && rsiValue >= Overbought)
            trade.PositionClose(sym);

         if(type == POSITION_TYPE_SELL && rsiValue <= Oversold)
            trade.PositionClose(sym);
      }
   }
}

//+------------------------------------------------------------------+
